# Generate Text From Your Model

Use beam search to select top 

In [1]:
import csv
import nltk
import random
from glove import Glove
import keras
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from random import randint
from pickle import load
import numpy as np
import tensorflow as tf
import re

/home/jamie/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text


In [3]:
confuc_model = Glove.load('Confuciembeddings1')

word_to_index = confuc_model.dictionary
word_to_vec_map = {word: confuc_model.word_vectors[confuc_model.dictionary[word]] for word in confuc_model.dictionary.keys()}

with open('Outputs/out_file1.csv') as csvfile:
    reader = csv.reader(csvfile)
    sentences2 = [item for sublist in reader for item in sublist]

sentences = [nltk.word_tokenize(sent) for sent in sentences2]
words = [item for sublist in sentences for item in sublist]

### Generate Sequences
flat_sequence = [word_to_index[i] for i in words]

length = 5 + 1
sequences = list()

for i in range(length, len(flat_sequence)):
    seq = flat_sequence[i - length: i]
    line = seq
    sequences.append(line)

In [4]:
word_to_vec_map['chi']

array([-0.08351763, -0.02707337,  0.06420108,  0.02763365, -0.03430993,
       -0.10714719, -0.02610115,  0.00942609,  0.01611823, -0.11255144,
        0.04171147, -0.01027372,  0.04588767,  0.12699698, -0.03430928,
       -0.03460288,  0.06490141, -0.04854851, -0.02558162, -0.05147224,
        0.02520145, -0.04340122,  0.03042588, -0.01438678,  0.00996038,
        0.0007495 ,  0.03615412, -0.03880004,  0.01456894,  0.03647816,
        0.03863734,  0.00556531,  0.0176869 , -0.01814223,  0.02674822,
       -0.03979636, -0.01830597, -0.00450005, -0.03189934,  0.04465651,
        0.0388217 , -0.04570852,  0.05225698,  0.03869111,  0.00717131,
       -0.01692131, -0.041752  ,  0.01237549, -0.03268834,  0.00530881,
       -0.05657496, -0.00933081,  0.03456618, -0.06184157, -0.03271414,
       -0.02675863,  0.03905797,  0.04186336,  0.02712894, -0.05819519,
       -0.03038191,  0.06709936, -0.0189233 , -0.09372057,  0.00250005,
        0.02532192, -0.08772955, -0.03102017, -0.02026014, -0.06

In [5]:
seq_length = len(sequences[0]) - 1

In [6]:
tf.reset_default_graph()

model = load_model('model.h10')

In [108]:
maxlen = 10

tokenizer = load(open('tokenizer2.pkl','rb'))

randomsentence = randint(0,len(words)-10)

random_sample = random.sample(range(0,len(words)-10),50)

seed_text_list = []

for i in random_sample:
    sentence = [word_to_index[i] for i in words[i:i+10]]
    seed_text_list.append(sentence)

sequence = np.array(seed_text_list[0])

print (seed_text_list[0])

[0, 1669, 39, 29, 51, 266, 128, 87, 2969, 1308]


In [109]:
def keras_rnn_predict(sequence, empty=0.0, rnn_model=model, maxlen=maxlen):
    """for each sample, calculate the probability for every possible label"""
    data = pad_sequences(sequence, maxlen=maxlen, value=empty)
    return rnn_model.predict(data, verbose=0)


In [110]:
def beamsearch(predict=keras_rnn_predict,k=1,maxsample=500,use_unk=False,oov=None,empty=0,eos=None):
    dead_k = 0
    dead_samples = []
    dead_scores = []
    live_k = 1
    live_samples = [[empty]]
    live_scores = [0]
    
    while live_k and dead_k < k:
        probs = predict(live_samples, empty=empty)
        
        cand_scores = np.array(live_scores)[:,None] - np.log(probs)
        if not use_unk and oov is not None:
            cand_scores[:,oov] = 1e20
        cand_flat = cand_scores.flatten()
        
        ranks_flat = cand_flat.argsort()[:(k-dead_k)]
        live_scores = cand_flat[ranks_flat]
        
        voc_size = probs.shape[1]
        
        live_samples = [live_samples[r//voc_size]+[r%voc_size] for r in ranks_flat]
        
        zombie = [s[-1] == eos or len(s) >= maxsample for s in live_samples]
        
        dead_samples += [s for s,z in zip(live_samples,zombie) if z]
        dead_scores += [s for s,z in zip(live_scores,zombie) if z]
        dead_k = len(dead_samples)
        
        live_samples = [s for s,z in zip(live_samples,zombie) if not z]
        live_scores = [s for s,z in zip(live_scores,zombie) if not z]
        live_k = len(live_samples)
        
    return dead_samples + live_samples, dead_scores + live_scores

In [111]:
ans = beamsearch(predict = keras_rnn_predict)

In [112]:
print(ans)

([[0, 121, 22, 680, 4, 499, 62, 238, 96, 62, 238, 51, 0, 1, 39, 96, 1391, 22, 2, 4, 882, 6, 1014, 100, 51, 96, 0, 321, 24, 322, 3, 2, 39, 96, 0, 321, 24, 322, 3, 3736, 1871, 1577, 12, 1339, 0, 4309, 4231, 6, 0, 4914, 12, 0, 3606, 0, 75, 24, 0, 1413, 12, 0, 2632, 12, 2293, 41, 843, 51, 0, 1, 39, 96, 0, 880, 155, 0, 684, 24, 0, 409, 27, 4, 114, 1708, 96, 9, 734, 2697, 141, 62, 238, 893, 51, 12, 1324, 0, 164, 6, 2915, 49, 3064, 6, 49, 227, 1774, 51, 0, 1, 39, 96, 0, 4206, 6, 0, 121, 6, 523, 0, 548, 24, 0, 684, 51, 0, 1, 39, 22, 27, 96, 1278, 8, 0, 621, 24, 62, 110, 22, 211, 4, 3087, 0, 3849, 24, 62, 238, 51, 0, 1, 39, 126, 9, 23, 2, 4, 96, 0, 2574, 24, 2699, 2, 123, 2081, 251, 64, 51, 224, 2, 670, 0, 121, 6, 114, 274, 96, 333, 155, 9, 1860, 24, 3657, 1951, 51, 0, 1, 39, 126, 22, 378, 4, 185, 6, 499, 62, 342, 12, 4, 4, 2504, 3, 2, 4, 2440, 6, 114, 137, 96, 9, 60, 23, 96, 62, 238, 16, 114, 2407, 96, 62, 77, 22, 680, 4, 1636, 62, 477, 51, 0, 1, 39, 96, 1391, 0, 624, 24, 0, 409, 23, 2, 2937, 

In [113]:
generated_paragraphs = []

for i in range(len(seed_text_list)):
    sequence = seed_text_list[i]
    preds, score = beamsearch(predict = keras_rnn_predict)
    preds = preds[score.index(max(score))]
    output = []
    for i in preds:
        out_word = ''
        for word, index in tokenizer.items():
            if index == i:
                out_word = word
                break
        output.append(out_word)
    sentences = ' '.join(output).split('.')
    sentences = sentences[1:len(sentences)-1]
    generated_paragraphs.append(sentences)
    print('progress ' + str(i) / str(len(seed_text_list)))

In [114]:
generated_sentences = [item for sublist in generated_paragraphs for item in sublist]

filename = open('confuc_gen.txt','w')

for item in generated_sentences:
    filename.write("%s\n" % item)

In [115]:
print(generated_sentences)

[' the master said in teaching he is not fit to converse others ', " in the book of poetry it is said in the book of poetry it needs round hands and shun the southern-wood ne'er to the chase and the yo the root of the river and the several and trees are held ", ' the master said in the bad as the kingdom of the mean who not be foregone in a minister situation after his conduct complaisance ', ' and keeping the prince to regulate their wills to their mode brothers ', " the master said in the announcement to the people to k'ang the duties of the kingdom ", ' the master said he who in excess with the princes of his country he does not dismiss the parent of his conduct ', " the master said when a man is not in the habit of saying-'what is proper wrong by what ", ' thus is because the people to be made in which as a position of institutions meat ', ' the master said when he had not equal to speak his point and not not forget it is not accordant to be good in a superior man in his conduct ha